In [4]:
#필요 라이브러리 호출
import os
import re
import json
import soynlp
import requests
from selenium import webdriver
from pykospacing import Spacing
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"
webdriver.ChromeOptions().add_argument("headless") 

In [12]:
#에펨코리아 단어별 상위 k개 페이지의 게시글 링크 수집
def get_links_fmko(search_word,pages=5):
    url="https://www.fmkorea.com/fifa_online"

    driver= webdriver.Chrome()
    driver.implicitly_wait(5)
    driver.get(url)

    driver.find_element(By.ID,'bd_srch_btm_itx_5665468').send_keys(search_word)
    driver.find_element(By.CLASS_NAME,'itx_wrp').find_element(By.TAG_NAME,'button').click()

    pages_key_url=driver.find_element(By.CLASS_NAME,'bd_pg').find_element(By.TAG_NAME,'a').get_attribute('href')[:-1]

    sele_urls=[]
    for sele in driver.find_elements(By.CLASS_NAME,"hx"):
        sele_urls.append(sele.get_attribute('href'))

    for i in range(2,pages+1):
        try:
            tmp_url=pages_key_url+str(i)

            driver.get(tmp_url)
            for sele in driver.find_elements(By.CLASS_NAME,"hx"):
                sele_urls.append(sele.get_attribute('href'))
        except:
            print("총 "+str(i-1)+"페이지가 검색되었습니다.")
            break
    driver.close()
    print("에펨코리아 페이지: 단어 "+search_word+"검색 완료")
    return sele_urls

def fmko_post_array(fmko_link):
    driver= webdriver.Chrome()
    driver.implicitly_wait(5)
    post_array=[]
    for i,tmp_url in enumerate(fmko_link):
        if i%20==0:
            driver.close()
            driver=webdriver.Chrome()
        try:
            driver.get(tmp_url)

            main_post=driver.find_element(By.ID,'bd_capture')
            head=main_post.find_element(By.CLASS_NAME,'np_18px').text
            date=main_post.find_element(By.CLASS_NAME,'date').text
            post=main_post.find_element(By.CLASS_NAME,'xe_content').text
            view=main_post.find_element(By.CLASS_NAME,'btm_area').find_element(By.CLASS_NAME,'fr').find_element(By.TAG_NAME,'span').text.split()[-1].replace(",","")
            try:
                recommend=driver.find_element(By.CLASS_NAME,'new_voted_count').text.replace(",","")
            except:
                recommend='0'
            try:
                comment_num=driver.find_element(By.CLASS_NAME,'fdb_tag').text.split()[1].replace(",","")
                comments=[]
                for comment in driver.find_elements(By.CLASS_NAME,'comment-content'):
                    try:
                        if comment.find_element(By.TAG_NAME,'a'):
                            comments.append(' '.join(comment.text.split()[1:]))
                    except:
                        comments.append(comment.text)
            except:
                comment_num='0'
                comments=[]
            post_array.append([head,date,post,view,recommend,comment_num,comments])
        except: ""
    driver.close()
    print("에펨코리아 페이지: 읽어오기 완료")
    return post_array

In [57]:
#커뮤니티 은어 정리 및 문장 정제
def word_changing(text_list):
    texts=text_list
    pattern1 = '[^\w]+[\W$]'
    pattern2 = '([ㄱ-ㅎㅏ-ㅣ]+)'

    texts = list(map(str, texts))                                                           # 숫자를 문자로 변환
    texts = list(map(lambda x: x.upper(),texts))
    texts = list(map(lambda x : x.replace(u"\n",u""), texts))                               # '\n' 삭제
    texts = list(map(lambda x : re.sub(pattern1," ",x), texts))                             # 기타 특수문자 제거

    texts =list(map(lambda x : x.replace('핵과금러',' 핵과금 사람'), texts))
    texts =list(map(lambda x : x.replace('중과금러',' 중과금 사람'), texts))
    texts =list(map(lambda x : x.replace('소과금러',' 소과금 사람'), texts))
    texts =list(map(lambda x : x.replace('무과금러',' 무과금 사람'), texts))

    texts =list(map(lambda x : x.replace('금카',' 금카 '), texts))

    texts =list(map(lambda x : x.replace('현질',' 과금'), texts))
    texts =list(map(lambda x : x.replace('선수 팩',' 선수팩'), texts))
    texts =list(map(lambda x : x.replace('월컵',' 월드컵'), texts))

    texts =list(map(lambda x : x.replace('날두',' 호날두'), texts))
    texts =list(map(lambda x : x.replace('호날드',' 호날두'), texts))
    texts =list(map(lambda x : x.replace('호날도',' 호날두'), texts))

    texts =list(map(lambda x : x.replace('라부',' 라이브퍼포먼스'), texts))
    texts =list(map(lambda x : x.replace('라이브부스트',' 라이브퍼포먼스'), texts))
    texts =list(map(lambda x : x.replace('라이브부스터',' 라이브퍼포먼스'), texts))
    texts =list(map(lambda x : x.replace('라이브 퍼포먼스','라이브퍼포먼스'), texts))

    texts =list(map(lambda x : x.replace('금고 소',' 소'), texts))
    texts =list(map(lambda x : x.replace('금고 중',' 중'), texts))
    texts =list(map(lambda x : x.replace('금고 대',' 대'), texts))

    texts =list(map(lambda x : x.replace('TOTS',' 토츠'), texts))
    texts =list(map(lambda x : x.replace('토스',' 토츠'), texts))
    texts =list(map(lambda x : x.replace('토즈',' 토츠'), texts))

    texts =list(map(lambda x : x.replace('비피',' BP'), texts))

    texts =list(map(lambda x : x.replace('콘',' ICON'), texts))
    texts =list(map(lambda x : x.replace('아이콘',' ICON'), texts))

    texts =list(map(lambda x : x.replace('감모',' 감독모드'), texts))

    texts =list(map(lambda x : x.replace('바칭코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('바칭코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('파칭코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('빠칭코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('빠징코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('빠찡코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('바찡코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('파찡코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('바징코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('파징코',' 빠칭코 '), texts))
    texts =list(map(lambda x : x.replace('빠친코',' 빠칭코 '), texts))

    texts =list(map(lambda x : x.replace('멤버십',' 멤버쉽 '), texts))
    texts =list(map(lambda x : x.replace('맴버십',' 멤버쉽 '), texts))
    texts =list(map(lambda x : x.replace('맴버쉽',' 멤버쉽 '), texts))

    texts =list(map(lambda x : x.replace('피방',' PC방'), texts))
    texts =list(map(lambda x : x.replace('피시방',' PC방'), texts))

    texts =list(map(lambda x : x.replace('피온',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('피좆',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('피파',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('좆파',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('FC 온',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('FC온',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('FC 온라인',' 피파온라인'), texts))
    texts =list(map(lambda x : x.replace('FC온라인',' 피파온라인'), texts))

    texts =list(map(lambda x : x.replace('피X',' 피파온라인X '), texts))
    texts =list(map(lambda x : x.replace('피파온라인 X',' 피파온라인X '), texts))


    texts =list(map(lambda x : x.replace('딸깍',' 딸깍팩 '), texts))
    texts =list(map(lambda x : x.replace('딸깍 팩',' 딸깍팩 '), texts))

    texts =list(map(lambda x : x.replace('큰 집',' 큰집 '), texts))
    texts =list(map(lambda x : x.replace('작은 집',' 작은집 '), texts))

    texts =list(map(lambda x : x.replace('상차',' 상자'), texts))

    texts =list(map(lambda x : x.replace('은메달',' 은메달상자 '), texts))
    texts =list(map(lambda x : x.replace('은메달 상자',' 은메달상자 '), texts))
    texts =list(map(lambda x : x.replace('은에달',' 은메달상자 '), texts))
    texts =list(map(lambda x : x.replace('은에달상자',' 은메달상자 '), texts))

    texts =list(map(lambda x : x.replace('무적권',' 무조건 '), texts))

    texts =list(map(lambda x : x.replace('열쇠 상자',' 열쇠상자 '), texts))

    texts =list(map(lambda x : x.replace('상자 깡',' 상자개봉 '), texts))
    texts =list(map(lambda x : x.replace('상자깡',' 상자개봉 '), texts))

    texts =list(map(lambda x : x.replace('부계',' 부캐 '), texts))
    texts =list(map(lambda x : x.replace('본계',' 본캐 '), texts))

    texts =list(map(lambda x : x.replace('구가',' 구단가치 '), texts))

    texts =list(map(lambda x : x.replace('대낙',' 대리낙찰 '), texts))
    texts =list(map(lambda x : x.replace('데낙',' 대리낙찰 '), texts))

    texts =list(map(lambda x : x.replace('강데',' 강화데이 '), texts))
    texts =list(map(lambda x : x.replace('강화 데이',' 강화데이 '), texts))

    texts =list(map(lambda x : x.replace('ㅇ',' 네'), texts))
    texts =list(map(lambda x : x.replace('ㄴ',' 아니오'), texts))

    texts = list(map(lambda x : re.sub(pattern2," ",x), texts))                             # 자음, 모음 제거
    texts = list(map(lambda x : x.replace("  "," "), texts))                                # 띄어쓰기 중복 제거
    texts = list(map(lambda x : x.strip(), texts))                                          # 문자 앞, 뒤 공백 제거

    texts=list(map(lambda x: soynlp.normalizer.repeat_normalize(x,num_repeats=1),texts))    # 반복되는 것 제거
    texts=list(map(lambda x: soynlp.normalizer.emoticon_normalize(x,num_repeats=1),texts))  # 이모티콘 제거

    return texts

In [53]:
#에펨코리아 주요단어 검색 및 게시글 링크 저장
fmko_link=[]
words=["현질","현금","결제","과금","유료"]
for word in words:
    fmko_link.append(get_links_fmko(search_word=word,pages=20))
fmko_link=sum(fmko_link,[])
fmko_link=list(set(fmko_link))

에펨코리아 페이지: 단어 현질검색 완료
에펨코리아 페이지: 단어 현금검색 완료
에펨코리아 페이지: 단어 결제검색 완료
에펨코리아 페이지: 단어 과금검색 완료
에펨코리아 페이지: 단어 유료검색 완료


In [15]:
#에펨코리아 게시글 링크에서 필요 내용 추출
fmko_posts=fmko_post_array(fmko_link)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.72)
Stacktrace:
	GetHandleVerifier [0x00007FF7F7609632+30946]
	(No symbol) [0x00007FF7F75BE3C9]
	(No symbol) [0x00007FF7F74B6FDA]
	(No symbol) [0x00007FF7F748CB85]
	(No symbol) [0x00007FF7F75337A7]
	(No symbol) [0x00007FF7F753A8C7]
	(No symbol) [0x00007FF7F752CA30]
	(No symbol) [0x00007FF7F74FA6E5]
	(No symbol) [0x00007FF7F74FB021]
	GetHandleVerifier [0x00007FF7F773F83D+1301229]
	GetHandleVerifier [0x00007FF7F774BDB7+1351783]
	GetHandleVerifier [0x00007FF7F7742A03+1313971]
	GetHandleVerifier [0x00007FF7F763DD06+245686]
	(No symbol) [0x00007FF7F75C758F]
	(No symbol) [0x00007FF7F75C3804]
	(No symbol) [0x00007FF7F75C3992]
	(No symbol) [0x00007FF7F75BA3EF]
	BaseThreadInitThunk [0x00007FF94F757374+20]
	RtlUserThreadStart [0x00007FF95093CC91+33]


In [10]:
# 리스트파일을 저장할 수 있도록 데이터 형태 정리
fmko_posting=[]
error=False
for i,post in enumerate(fmko_posts):
    tmp=word_changing(post[:6])
    comment_tmp=word_changing(post[-1])
    tmp.append(comment_tmp)
    fmko_posting.append(tmp)
if error: print("포스트 중 일부가 너무 깁니다.")

NameError: name 'fmko_posts' is not defined

In [59]:
# 추출 내용 저장 코드
with open('fmko_list.txt', 'w') as file:
    for post_tmp in fmko_posting:
        comments=post_tmp[-1]
        tmp=post_tmp[:6]
        txt="//구분자//".join(map(str,tmp))
        file.write(f"{txt}\n")
        txt="//구분자//".join(comments)
        file.write(f"{txt}\n")

# 추출 게시글 링크 저장 코드
with open('fmko_link.txt', 'w') as file:    
    txt="\n".join(fmko_link)
    file.write(f"{txt}\n")